In [ ]:
# --- Import the Salary Data ---

import pandas as pd
from pathlib import Path

# Define path to data folder and CSV file
DATA_PATH = Path("../data")
CSV_FILE = DATA_PATH / "Salaries.csv"

print("Setup complete. CSV file path:", CSV_FILE)


In [ ]:
# --- Function to Load Salary Data ---

class DataLoadError(Exception):
    """Custom error raised when salary data cannot be loaded."""
    pass

def load_salary_data(csv_path: Path) -> pd.DataFrame:
    """
    Reads the salary CSV file and returns it as a pandas DataFrame.
    Includes basic error handling for missing or unreadable files.
    """
    if not csv_path.exists():
        raise DataLoadError(f"⚠️ File not found: {csv_path}")
    
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        raise DataLoadError(f"❌ Could not read CSV file. Error: {e}")
    
    print(f"✅ Successfully loaded {len(df)} records from '{csv_path.name}'")
    return df


In [ ]:
# --- Load the data and show a preview ---

df = load_salary_data(CSV_FILE)
df.head()